In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

In [2]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    shakespeare = f.read()
    
# List all unique characters that occurs in the input text
chars = sorted(list(set(shakespeare)))
vocab_size = len(chars)

In [3]:
# mapping from characters to integers for encoding
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s] # Take a string and output a list of integers
decode = lambda i : ''.join([itos[c] for c in i]) # Take a list of integers and output a list of string

In [4]:
# encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(shakespeare), dtype=torch.long)

In [5]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will be process in parallel?
block_size = 8 # What is the maximum context length for predictions?

In [7]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [8]:
class Head(nn.Module):
    """ One head of self-attention """
    
    def __init__(self, n_embd, block_size, head_size) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril((torch.ones(block_size, block_size))))
    
    def forward(self, x):
        B, T, C = x.shape
        
        # Compute the attention score using k and q 
        k = self.key(x)
        q = self.query(x)
        w = q @ k.transpose(-2, -1) * C ** -0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        
        # weight aggregation of the values using the attention score
        v = self.value(x)
        out = w @ v
        
        return out

In [9]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, n_embd, block_size, head_size, n_head):
        super().__init__()
        self.head = nn.ModuleList([Head(n_embd, block_size, head_size) for _ in range(n_head)])
    
    def forward(self, x):
        # Use multiple heads of self-attention 
        # Concatanate each output alone the embedding dimension (B, T ,C), that is merged alone C
        return torch.cat([h(x) for h in self.head], dim=-1)

In [10]:
# A modified neural network for bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, block_size, n_head) -> None:
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # ------------------modified--------------------------------------
        # use multiple head attention
        self.multi_attn_head = MultiHeadAttention(n_embd, block_size, n_embd//n_head, n_head)
    
        # ------------------modified--------------------------------------

        self.ln_head = nn.Linear(n_embd, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = tok_emb + pos_emb
        
        # ------------------modified--------------------------------------
        x = self.multi_attn_head(x)
        # ------------------modified--------------------------------------

        logits = self.ln_head(x)            
        
        if targets is None: # for generation without providing target
            loss = None
        else:
            # Reshape the logits tensor to meet definition of the cross_entropy function in Pytorch
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Concatenate B, T
            targets = targets.view(B*T)    # Same reshaping to the target tensor
            loss = F.cross_entropy(logits, targets) # calculate the loss
        
        return logits, loss
    
    def generation(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens to meet position embedding range
            idx_crop = idx[:, -self.block_size:]
            
            # get the predictions
            logits, _ = self(idx_crop)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx

In [11]:
model = BigramLanguageModel(vocab_size, n_embd=32, block_size=block_size, n_head=4)

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [13]:
from tqdm import tqdm

batch_size = 32
epochs = 10000
losses = []
for e in tqdm(range(epochs)):
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

100%|██████████| 10000/10000 [00:42<00:00, 234.25it/s]


In [14]:
print(loss.item())

2.1647799015045166


In [15]:
print(decode(model.generation(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


hadd my thiot sotherea.

On min etared cons?

Isomens.

Thathis afpich thake ay ind, cigig, un bec! that monnoth my
counge ipponn I ou madd belt anp'y st bazel
The ward, de's word'mpselarmid.

DUS:
Is'ails seling is whee,
Koresth's'at, blects ance. De! as-'ll nef, coof thes 'dis aswalt honge?

BYor ley:
Of coust woust,
Gons obleearsuldy, mintrefepetne unre not
YaSumay out sit suple;
Dhave nellfor,
Ay to bolkervan my do ther.

Dive se in afill waghtlee the kener bors ow clis unkens meeme-werags
W
